# ***Big Query API Setup***

In [ ]:
!pip install google-cloud-bigquery

In [ ]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/centering-river-378911-2832f5da35cf.json'

In [ ]:
from google.cloud import bigquery

# Explicitly create a client object
client = bigquery.Client()

# Or, if you've set the GOOGLE_APPLICATION_CREDENTIALS environment variable,
# you can create the client without explicitly passing credentials:
# client = bigquery.Client()



In [ ]:
# Example query
query_job = client.query("SELECT * FROM `bigquery-public-data.covid19_open_data.covid19_open_data` LIMIT 10")
results = query_job.result()

for row in results:
    print(row)

In [ ]:
df = query_job.to_dataframe()
df.head()

#***Advanced SQL with BIG Query***

In [ ]:
sql = """     """ + " limit 10"

In [ ]:
df = client.query(sql).to_dataframe()
df.head()

In [ ]:
#Searching A DataBase In Big Query
data_base = client.dataset('hacker_news',project="bigquery-public-data")

In [ ]:
dataset = client.get_dataset(data_base)
dataset

In [ ]:
for i in client.list_tables(dataset):
  print(i.dataset_id)

In [ ]:
client = bigquery.Client()
def query(sql):
  df = client.query(sql).to_dataframe()
  return df


In [ ]:
query(
    """select *
       from bigquery-public-data.hacker_news.full
       limit 10"""
)

In [ ]:
sql = """
with cte as(
  select id,extract(month from min(timestamp)) as min_month
  from bigquery-public-data.hacker_news.full
  where timestamp is not null
  group by id
)
select
extract(month from h.timestamp) as month,
count(case when extract(month from h.timestamp) >= c.min_month then 1 end) as count_mis,
count(*) as percentage
from bigquery-public-data.hacker_news.full h
left join cte c on h.id = c.id
group by extract(month from h.timestamp)


"""


query(sql)

In [ ]:
!pip install duckdb

In [ ]:
import pandas as pd
import duckdb

In [ ]:
data = pd.read_csv("https://d2beiqkhq929f0.cloudfront.net/public_assets/assets/000/039/649/original/olympics.csv?1689050368")
data.head()

In [ ]:
# pd.DataFrame([[1, 2], ["Ram", "Shyam"], ["IT", "Ops"]], columns = ["emp_id", "name", "dept"])
pd.DataFrame({'emp_id':[1, 2], 'name': ['Ram', 'Shyam'], 'dept':['IT', 'Ops']})

In [ ]:
pd.DataFrame([[1, "Ram", "IT"], [2, "Shyam", "Ops"]], columns = ["emp_id", "name", "dept"])

In [ ]:
data.loc[:3,'City':'Gender']

In [ ]:
data.loc[:,['City','Sport','Edition']]

In [ ]:
data.nunique()

In [ ]:
pd.DataFrame(data,columns=['City','Gender'])

In [ ]:
sql = """select count(distinct trim(Event)) from data"""

duckdb.query(sql)

In [ ]:
sql = """select count(distinct Event) from data
where Event like '100m freestyle%'"""

duckdb.query(sql)

In [ ]:
577-145

In [ ]:
sql = """select City
from data
where Sport = 'Cricket'"""

duckdb.query(sql)

In [ ]:
data.head(2)

In [ ]:
sql = """select Gender,count(*) as count_g
from data
where Medal='Gold'
group by Gender"""

duckdb.query(sql)

In [ ]:
sql = """select City, count(*)
from (select City,Edition
from data
group by City,Edition)t
group by City
having count(*) > 1
"""

duckdb.query(sql)

In [ ]:
#Beijing
sql = """select NOC,count(*)
from data
where City = 'Beijing'
group by 1
order by count(*) desc
"""

duckdb.query(sql)


In [ ]:
#Avg Medals For Each Country
sql = """select avg(m_sum)
from (select NOC,sum(Count_of_medals) as m_sum
from data
where Gender = 'Men'
group by 1)t
"""

duckdb.query(sql)

In [ ]:
#Women Athlets
sql = """select Athlete,sum(Count_of_medals) as m_sum
from data
where Gender = 'Women' and NOC='USA'
group by 1
order by 2 desc
"""

duckdb.query(sql)

In [ ]:
data['Sport'].unique()

In [ ]:
#Most Medals in Gymnastics in the year 1948 to 1992
sql = """select NOC,sum(Count_of_medals) as m_sum
from data
where Sport = 'Gymnastics' and Edition between 1948 and 1992
group by 1
order by 2 desc
"""

duckdb.query(sql)


In [ ]:
data.head()

## #SQl Practice

In [ ]:
# prompt: write a code to connect to drive and read a csv file in it

from google.colab import drive
drive.mount('/content/drive')
# Replace 'your_file.csv' with the actual name of your CSV file
# and the path to it within your Google Drive
file_path = '/content/drive/My Drive/your_file.csv'


In [ ]:
import pandas as pd
import duckdb

In [ ]:
!wget 'https://drive.google.com/drive/folders/1fE3bCcsNiDdUqDKNBVqhdzXbop6nwrnb' employees.csv

In [ ]:
employees = pd.read_csv('employee_base.csv')
employees.head()

In [ ]:
sql = """with cte as(select EmployeeID,Category,Lead(EmployeeID,1,9999) over(order by EmployeeID) num  from employees where Category is not null )

select * from employees e join cte c on e.EmployeeID between c.EmployeeID and c.num-1
"""

duckdb.query(sql)

In [ ]:
sql = """with recursive cte as ((select cast('2022-01-03' as date) as level)
union all
(select date_add(level,interval 1 day) from cte where level<'2022-03-15'))
select * from cte
"""

duckdb.query(sql)

In [ ]:
sql = """ drop table employee;
create table employee ( emp_id int, company varchar(10), salary int );

Insert into employee values (1,'A',2341);insert into employee values (2,'A',341); insert into employee values (3,'A',15); insert into employee values (4,'A',15314); insert into employee values (5,'A',451); insert into employee values (6,'A',513); insert into employee values (7,'B',15); insert into employee values (8,'B',13); insert into employee values (9,'B',1154); insert into employee values (10,'B',1345); insert into employee values (11,'B',1221); insert into employee values (12,'B',234); insert into employee values (13,'C',2345); insert into employee values (14,'C',2645); insert into employee values (15,'C',2645); insert into employee values (16,'C',2652); insert into employee values (17,'C',65);
select * from employee;
"""

duckdb.query(sql)

In [ ]:
sql = """with cte as(select *,row_number() over(partition by company order by salary) sort,count(*) over(partition by company) maxi from employee)
,cte3 as(select *,if(maxi%2=0,if(cast(maxi/2 as int)=sort or cast(maxi/2 as int)=sort-1,1,0),if(cast(maxi/2 as int)=sort-1,1,0)) abc from cte)
select company,avg(if(abc=1,salary,null)) from cte3 group by company;
"""
duckdb.query(sql)

In [ ]:
sql = """with cte as(select *,row_number() over(partition by company order by salary) sort,count(*) over(partition by company) maxi from employee)
,cte3 as(select *,if(maxi%2=0,if(cast(maxi/2 as int)=sort or cast(maxi/2 as int)=sort-1,1,0),if(cast(maxi/2 as int)=sort-1,1,0)) abc from cte)
--select company,avg(if(abc=1,salary,null)) from cte3 group by company;
select * from cte3
"""
duckdb.query(sql)

In [ ]:
sql = """with cte as(select *,row_number() over(partition by company order by salary) sort,count(*) over(partition by company) count_s from employee)
select company,avg(salary) from cte where sort in ((count_s+1)/2,(count_s+2)/2,(count_s)/2)
group by company
"""
duckdb.query(sql)

In [ ]:
sql="""
create table stadium ( id int, visit_date date, no_of_people int );
insert into stadium values (11,'2017-07-01',10) ,(21,'2017-07-02',109) ,(13,'2017-07-03',150) ,(14,'2017-07-04',99) ,(25,'2017-07-05',145) ,(96,'2017-07-06',1455) ,(87,'2017-07-07',199) ,(98,'2017-07-08',188);
"""
duckdb.query(sql)

In [ ]:
sql="""select * from stadium
union all
select * from stadium where visit_date = (select max(visit_date) from stadium)"""
duckdb.query(sql)

In [ ]:
sql="""with cte1 as(select *,if(no_of_people > 100,1,0) flag from stadium)
,cte as (select *,sum(flag) over(order by visit_date rows between 2 preceding and current row) sum_ from cte1)
select * from cte"""
duckdb.query(sql)

In [ ]:
sql="""with cte1 as(select *,if(no_of_people > 100,1,0) flag from stadium)
,cte as (select *,lead(flag,1,1) over(order by visit_date) next_,lead(flag,2,1) over(order by visit_date) prev from cte1)
select * from cte where (flag+next_+prev)>=3"""
duckdb.query(sql)

In [ ]:
sql="""with tabcte as(select * from stadium union all select * from stadium where id=98 union all select * from stadium where visit_date>='2017-07-07')
select s1.visit_date,group_concat(s2.no_of_people)
 from tabcte s1 left join tabcte s2 on s2.visit_date between s1.visit_date and date_add(s1.visit_date, interval 2 day) and s2.no_of_people > 100
group by s1.visit_date having count(*)>2

"""
duckdb.query(sql)

In [ ]:
sql="""with tabcte as(select * from stadium union all select * from stadium where id=98 union all select * from stadium where visit_date>='2017-07-07')"""
duckdb.query(sql)

In [ ]:
products = pd.read_csv('/content/drive/MyDrive/Data Scince/Northwind Traders/products.csv')
products.head()

In [ ]:
order_details = pd.read_csv('/content/drive/MyDrive/Data Scince/Northwind Traders/orders_details.csv')
order_details.head()

In [ ]:
order_details[order_details['productid']==72]['discount']

In [ ]:
products[products['productname']=='Mozzarella di Giovanni']

In [ ]:
sql="""select sum((1-o.discount)*o.unitprice) from order_details o left join products p using(productid)
where p.productname='Mozzarella di Giovanni' group by o.productid"""
duckdb.query(sql)

In [ ]:
orders = pd.read_csv('/content/drive/MyDrive/Data Scince/Northwind Traders/orders.csv')
customers = pd.read_csv('/content/drive/MyDrive/Data Scince/Northwind Traders/customers.csv')

sql="""
select c.customerid from customers c left join orders o on c.customerid = o.customerid and o.employeeid = 4 where o.customerid is null"""
duckdb.query(sql)

In [ ]:
sql="""with cte as(select *,if(((unitsinstock + unitsonorder)<=reorderlevel) and discontinued=0,1,0) flag from products)
select productname from cte where flag=1 and productname like 'Outback_Lager'"""
duckdb.query(sql)
# products.head()